In [2]:
import pandas as pd
import numpy as np
import urllib2
from bs4 import BeautifulSoup
import re

In [3]:
df = pd.read_csv('Regional_links.csv')

In [4]:
df.head()

,region,type,link
0,4553,0,http://www.cian.ru/sale/commercial/13973069/
1,4553,0,http://www.cian.ru/sale/commercial/14031534/
2,4553,0,http://www.cian.ru/sale/commercial/151524952/
3,4553,1,http://www.cian.ru/sale/commercial/149933741/
4,4553,1,http://www.cian.ru/sale/commercial/8978344/


In [11]:
def parse_page(page=''):
    # parse pages for data
    try:
        req = urllib2.Request(page)
        response = urllib2.urlopen(req)
        the_page = response.read()
        bs = BeautifulSoup(the_page, 'html.parser')
        print('page',page,'works fine')
        return bs
    except:
        print('something goes wrong at',page)

In [ ]:
%%time
reg = re.compile('\"center\": \[([0-9\.]+)\, ([0-9\.]+)\]')
result = []
for number,link in enumerate(list(df.link.unique())):
    if number%1000==0 and number!=0:
        pd.DataFrame(result).to_csv('regions_res{}.csv'.format(number),index=False,encoding='utf8')
    try:
        bs = parse_page(page=link)

        for i in bs.find_all('meta', attrs={'itemprop':'price'}):
            #price = i.attrs['content']
            result.append([link, 'price', i.attrs['content']])
        for i in bs.find_all('meta', attrs={'property':'og:description'}):
            #desc = i.attrs['content']
            result.append([link, 'desc', i.attrs['content']])
        for i in bs.find_all('script'):
            k = reg.search(i.text)
            if k:
                result.append([link, 'coordinates', str(k.group(1))+','+str(k.group(2))])

        name = []
        value = []    
        for i in bs.find_all('dt'):
            name.append(i.text.encode('utf8').strip())  
        for i in bs.find_all('dd'):
            value.append(i.text.encode('utf8').strip())
        for n,i in enumerate(name):
            result.append([link,i,value[n]])
    except:
        pass

('page', 'http://www.cian.ru/sale/commercial/13973069/', 'works fine')
('page', 'http://www.cian.ru/sale/commercial/14031534/', 'works fine')
('page', 'http://www.cian.ru/sale/commercial/151524952/', 'works fine')


In [ ]:
pd.DataFrame(result,columns=['link','feature','value']).to_csv('regions_res_all.csv',index=False,encoding='utf8')

In [89]:
%%time
for i in bs.find_all('meta', attrs={'itemprop':'price'}):
    print i.attrs['content']

9300000,0
Wall time: 34 ms


In [88]:
%%time
for i in bs.find_all('meta', attrs={'property':'og:description'}):
    print i.attrs['content']

Предлагаю к продаже помещение свободного назначения на 1 этаже жилого дома (первая линия, отдельный вход со двора), находящемся по адресу: Москва, улица Люблинская, дом 127/1. <br />Помещение идеально подходит под размещение магазина (в том числе получение алкогольной лицензии), салона красоты или детского развивающего центра. Помещение имеет центральные коммуникации, включая энергоснабжение, отопление, холодную воду, канализацию, МГТС. Помещение оборудовано стеклопакетами и электрическими жалюзи. Хорошее состояние, высокие потолки (более 4 метров), свежий ремонт, готовность к въезду. Кабинетная система. Возможность перепланировки и объединения в общий зал с организацией дополнительного входа. Дом находится в плотном секторе жилой застройки, обеспечивая хорошую транспортную и пешеходную доступность.
Wall time: 31 ms


In [96]:
%%time
for i in bs.find_all('dt'):
    print i.text.encode('utf8').strip()  
for i in bs.find_all('dd'):
    print i.text.encode('utf8').strip()                       

Площадь:
Этаж:
Высота потолков:
Помещение:
Специализация помещения:
Состояние:
Тип здания:
Общая площадь:
Участок:
Категория:
Вентиляция:
Кондиционирование:
Отопление:
Система пожаротушения:
74 м²
1 из 6
4 м
свободно
бытовые услуги, 
         
      
        
          кальянная, 
         
      
        
          клуб, 
         
      
        
          коммерцию, 
         
      
        
          магазин, 
         
      
        
          продукты, 
         
      
        
          салон красоты, 
         
      
        
          стоматология
типовой ремонт
жилой дом
74,4 м²
в собственности
действующий
нет
нет
центральное
нет
Wall time: 5 ms


In [70]:
#reg = re.compile('\"center\": \[*\]')
reg = re.compile('\"center\": \[([0-9\.]+)\, ([0-9\.]+)\]')

In [75]:
%%time
for i in bs.find_all('script'):
    k = reg.search(i.text)
    if k:
        print k.group(1),k.group(2)

55.672818 37.740228
Wall time: 5 ms
